In [ ]:
# default_exp chat

# get

> API details.

In [1]:
#export
import os, openai, requests
from awsSchema.apigateway import Event, Response
from copy import deepcopy
from beartype import beartype
from nicHelper.secrets import getSecret
from openai import Completion

In [2]:
#export
class ParseInputError(Exception):
    pass

In [17]:
#export
EventInput = dict

@beartype
def parseEvent(event):
    eventCopy = deepcopy(event)
    body = Event.parseBody(eventCopy)

    try:
        inputText = body['inputText']
    except KeyError:
        raise ParseInputError('inputText is not in body')
    
    return inputText

@beartype
def talkWithOpenAI(inputText: str)->str:
    openai.api_key = getSecret('openai')['key']
    r = Completion.create(engine="davinci", prompt=inputText, max_tokens=100, temperature=0.2)
    # headers = {"Authorization":f"Bearer {api_key}"}
    # event = {"model": "text-davinci-003", "prompt": inputText, "temperature": 0.2, "max_tokens": 100}
    # print('input text is', inputText)
    # r = requests.post('https://api.openai.com/v1/completions', headers=headers, json=event)
    print("r: ", r)

    return r["choices"][0]["text"]

In [18]:
#export
def chat(event, *args):
    try:
        inputText = parseEvent(event)
        response = talkWithOpenAI(inputText)
        return Response.returnSuccess(body = response)
    except Exception as e:
        return Response.returnError(message=str(e))

In [19]:
#hide
import json
inputSample = {"inputText": "Hello my name is sam. What is your favorite thinga about me?"}
input_ = Event.getInput(inputSample)
print("\n input is", input_, "\n")
x = chat(input_)
print("body = ", x)


 input is {'body': '{"inputText":"Hello my name is sam. What is your favorite thinga about me?"}', 'httpMethod': None, 'multiValueHeaders': None, 'multiValueQueryStringParameters': None, 'path': None, 'pathParameters': None, 'queryStringParameters': {}, 'requestContext': None, 'headers': {}, 'statusCode': 200, 'isBase64Encoded': False} 

r:  {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nI love your eyes.\n\nI love your smile.\n\nI love your hair.\n\nI love your voice.\n\nI love your laugh.\n\nI love your personality.\n\nI love your body.\n\nI love your personality.\n\nI love your eyes.\n\nI love your smile.\n\nI love your hair.\n\nI love your voice.\n\nI love your laugh.\n\nI love your personality.\n\n"
    }
  ],
  "created": 1675752305,
  "id": "cmpl-6hC0v8LAkViXE9k2OVAidEL8OR2DV",
  "model": "davinci",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 15,
    "total_t